## Welcome to BQuant
<img src="../images/workflow.png">

# Sheet Navigation

<a href = "#A">Initial Examples</a> 

<a href = "#B">Next Example</a> 

<a href = "#Z">BDH Style Data Fetch</a>  

<br>


In [1]:
import bql
import pandas as pd

bq = bql.Service()

### Fetch some data  <a name="A"></a>


In [2]:

request = """
get(PX_LAST, PX_VOLUME, PX_LOW) 
for(['TSLA US Equity', 'ABX CN Equity']) 
with(start=-60d,end=0d,fill='prev')
"""

response = bq.execute(request)

# create dataframe: combined_df will 'stack' multiple fields instead doing a join
newdf =bql.combined_df(response)    

newdf.sample(15)

,DATE,CURRENCY,PX_LAST,PX_VOLUME,PX_LOW
ID,,,,,
ABX CN Equity,2020-06-14,None,NaN,7750275.0,NaN
TSLA US Equity,2020-06-06,None,NaN,7811917.0,NaN
TSLA US Equity,2020-04-19,USD,753.890000,NaN,747.660000
TSLA US Equity,2020-06-07,None,NaN,7811917.0,NaN
TSLA US Equity,2020-06-18,USD,1000.314575,NaN,994.470825
ABX CN Equity,2020-04-19,CAD,34.400000,NaN,32.550000
TSLA US Equity,2020-06-16,None,NaN,14051078.0,NaN
TSLA US Equity,2020-06-01,None,NaN,15085297.0,NaN
TSLA US Equity,2020-05-24,USD,816.880000,NaN,812.000000


In [5]:
from functools import reduce

df_l = [x.df().reset_index() for x in list(response)]
newdf3 = reduce(lambda left,right: pd.merge(left,right,on=['ID', 'DATE']), df_l)

In [6]:
newdf3.drop(['CURRENCY_y', 'CURRENCY_x'], axis=1, inplace=True)
newdf3.head()

,ID,DATE,PX_LAST,PX_VOLUME,PX_LOW
0,TSLA US Equity,2020-04-19,753.89,13128237.0,747.66
1,TSLA US Equity,2020-04-20,746.36,14746577.0,712.21
2,TSLA US Equity,2020-04-21,686.72,20209093.0,673.79
3,TSLA US Equity,2020-04-22,732.11,14224831.0,688.71
4,TSLA US Equity,2020-04-23,705.63,13236697.0,703.13


In [7]:
#newdf3.set_index('DATE', inplace=True)
#newdf4_ = newdf3.filter(like='PX', axis=1)
#newdf4_['DATE'] = newdf3['DATE']

In [8]:
#newdf4_.sample(5)

In [9]:
#newdf4 = newdf3.pivot(index='DATE', columns='ID',values = 'PX_LAST')
newdf4 = newdf3.pivot(index='DATE', columns='ID')
newdf4.head()

PX_LAST                    PX_VOLUME                 \
ID         ABX CN Equity TSLA US Equity ABX CN Equity TSLA US Equity   
DATE                                                                   
2020-04-19         34.40         753.89     8156432.0     13128237.0   
2020-04-20         35.24         746.36     7884899.0     14746577.0   
2020-04-21         35.16         686.72     7685389.0     20209093.0   
2020-04-22         37.57         732.11     8036314.0     14224831.0   
2020-04-23         37.65         705.63    13576251.0     13236697.0   

                  PX_LOW                 
ID         ABX CN Equity TSLA US Equity  
DATE                                     
2020-04-19         32.55         747.66  
2020-04-20         34.00         712.21  
2020-04-21         34.19         673.79  
2020-04-22         36.15         688.71  
2020-04-23         37.40         703.13

In [10]:
#newdf4.head().style.set_caption('PX_LAST')


In [11]:
import bqviz as bqv

# Pass the dataframe to LinePlot and apply Bloomberg styles
ex1 = bqv.InteractiveLinePlot(newdf4['PX_LAST'])

# Display the plot
ex1.show()

from functools import reduce

request = """
get(PX_LAST, PX_VOLUME) 
for(['EEM US Equity', 'TSLA US Equity']) 
with(start=-60d,end=0d,fill='prev')
"""

response = bq.execute(request)

### create dataframe: combined_df will 'stack' multiple fields instead doing a join. Use if single field for a list of tickers.
#newdf =bql.combined_df(response)    

### create dataframe: list comprehension will staple columns together. Use for multiple fields (no history)
#newdf = pd.concat([x.df() for x in response], axis=1)

### create dataframe: use 'merge' and 'reduce' to effectively join. Use for multiple fields and hisotrical data
df_l = [x.df().reset_index() for x in list(response)]
newdf = reduce(lambda left,right: pd.merge(left,right,on=['ID', 'DATE']), df_l)

newdf

In [4]:
#Function to convert the result of a BQL query to a normal pandas dataframe
from functools import reduce

def merge_dfs(response):
    df_l = [x.df().reset_index() for x in list(response)]
    newdf = reduce(lambda left,right: pd.merge(left,right,on=['ID', 'DATE']), df_l)
    return newdf

data = merge_dfs(response)

In [5]:
data.sample(7)

,ID,DATE,CURRENCY_x,PX_LAST,PX_VOLUME,CURRENCY_y,PX_LOW
117,ABX CN Equity,2020-05-10,CAD,38.13,6558635.0,CAD,37.960
78,ABX CN Equity,2020-04-01,CAD,26.88,8211981.0,CAD,25.850
32,TSLA US Equity,2020-04-16,USD,745.21,20657862.0,USD,706.715
23,TSLA US Equity,2020-04-07,USD,545.45,17919784.0,USD,532.340
42,TSLA US Equity,2020-04-26,USD,725.15,13237612.0,USD,698.180
7,TSLA US Equity,2020-03-22,USD,427.53,28285502.0,USD,425.790
48,TSLA US Equity,2020-05-02,USD,701.32,32531807.0,USD,683.040


In [11]:
newdf.shape

(244, 5)

In [12]:
data.shape

(122, 7)

### Put dataframe in easily chartable format
> Reindex on datetime

In [13]:
#newdf = newdf.reset_index()
#data = data.reset_index()
#del newdf['CURRENCY']
#newdf
data.sample(20)

,ID,DATE,CURRENCY_x,PX_LAST,PX_VOLUME,CURRENCY_y,PX_LOW
111,CL US Equity,2020-02-01,USD,73.780000,13592326.0,USD,73.370
51,TSLA US Equity,2020-02-02,USD,650.570000,15719266.0,USD,632.520
23,TSLA US Equity,2020-01-05,USD,443.010000,17794697.0,USD,436.920
31,TSLA US Equity,2020-01-13,USD,524.860000,26634547.0,USD,492.000
115,CL US Equity,2020-02-05,USD,75.730000,4282939.0,USD,74.510
59,TSLA US Equity,2020-02-10,USD,771.280000,24689163.0,USD,752.400
92,CL US Equity,2020-01-13,USD,70.140000,4641349.0,USD,69.800
47,TSLA US Equity,2020-01-29,USD,580.990000,18216672.0,USD,567.430
58,TSLA US Equity,2020-02-09,USD,748.070000,17063521.0,USD,730.000
19,TSLA US Equity,2020-01-01,USD,418.330000,10292456.0,USD,402.080


In [14]:
import pandas

data2 = data.pivot(index ='DATE', columns='ID', values = 'PX_LAST')
data2.head(15)

ID,CL US Equity,TSLA US Equity
DATE,,
2019-12-13,68.69,358.39
2019-12-14,68.69,358.39
2019-12-15,68.69,358.39
2019-12-16,68.87,381.50
2019-12-17,67.84,378.99
2019-12-18,68.27,393.15
2019-12-19,68.34,404.04
2019-12-20,68.85,405.59
2019-12-21,68.85,405.59


In [15]:
# Import the bqviz and bqplot libraries
import bqviz as bqv

# Pass the example DataFrame to the bqviz LinePlot object and apply Bloomberg styles
bqv_plot = bqv.LinePlot(data2).set_style()

# Display the plot
bqv_plot.show()

GridBox(children=(Figure(animation_duration=500, axes=[Axis(color='white', grid_color='#3c3c3c', grid_lines='d…

<div style="text-align: right"><a href="#Sheet-Navigation">Return to Top ↑</a></div>

## Next Example <a name="B"></a>

In [ ]:
request2 = """
GET(NAME)
FOR(filter(members('SPX Index'),GICS_SECTOR_NAME == 'Financials'))
"""


response = bq.execute(request2)

newdf =bql.combined_df(response)    # newdf = response[0].df() if only one field
newdf

In [ ]:
type(response)


# Eco Example #1

In [ ]:
request3 = """
get(GDP(PO=range(-7Q,0Q)))
for(['US Country','GB Country'])
"""

response = bq.execute(request3)

df =bql.combined_df(response)    # newdf = response[0].df() if only one field
df.head(10)

# Eco Example #2

In [2]:
letstring = """
let(
#PMI = PMI(PO=range(-35M,0M));
#ConConf = Consumer_Confidence(PO=range(-35M,0M));
#BusConf = Business_Confidence(PO=range(-35M,0M));
#CPI = CPI(PO=range(-35M,0M));
#PPI = PPI(PO=range(-35M,0M));
#RetailSales = Retail_Sales(PO=range(-35M,0M));
)
"""

getstring = """
get(#pmi, #ConConf, #BusConf,#CPI, #PPI,#RetailSales)
"""

country = "\'GB Country\'"


forstring = "for" + "("+ country + ")"

request5 = letstring + getstring + forstring

In [30]:
response = bq.execute(request5)

df =bql.combined_df(response)    # newdf = response[0].df() if only one field
df.sample(15)

,PERIOD_REFERENCE_DATE,REPORTED_DATE,CONCEPT,ECO_SOURCE,PERIOD,PERIOD_END_DATE,SEASONALITY,#pmi,PERIOD_OVER_PERIOD,TRANSFORM,#ConConf,#BusConf,#CPI,#PPI,#RetailSales
ID,,,,,,,,,,,,,,,
GB Country,2019-11-30,2019-11-28,CONSUMER_CONFIDENCE,GfK NOP (UK),2019M11,2019-11-30,NSA,NaN,None,None,-14.0,NaN,NaN,NaN,NaN
GB Country,2017-11-30,2017-12-01,PMI,Markit,2017M11,2017-11-30,SA,58.2,None,None,NaN,NaN,NaN,NaN,NaN
GB Country,2018-01-31,NaT,BUSINESS_CONFIDENCE,European Commission,2018M1,2018-01-31,SA,NaN,None,None,NaN,-7.5,NaN,NaN,NaN
GB Country,2019-05-31,2019-06-20,RETAIL_SALES,UK Office for National Statistics,2019M5,2019-05-31,SA,NaN,YOY,PCT,NaN,NaN,NaN,NaN,2.1
GB Country,2018-01-31,2018-02-16,RETAIL_SALES,UK Office for National Statistics,2018M1,2018-01-31,SA,NaN,YOY,PCT,NaN,NaN,NaN,NaN,2.0
GB Country,2020-04-30,2020-05-08,CONSUMER_CONFIDENCE,GfK NOP (UK),2020M4,2020-04-30,NSA,NaN,None,None,-34.0,NaN,NaN,NaN,NaN
GB Country,2019-09-30,2019-10-16,CPI,UK Office for National Statistics,2019M9,2019-09-30,NSA,NaN,YOY,PCT,NaN,NaN,1.7,NaN,NaN
GB Country,2018-02-28,2018-03-01,PMI,Markit,2018M2,2018-02-28,SA,55.3,None,None,NaN,NaN,NaN,NaN,NaN
GB Country,2018-12-31,2019-01-16,PPI,UK Office for National Statistics,2018M12,2018-12-31,NSA,NaN,YOY,PCT,NaN,NaN,NaN,2.40857,NaN


In [34]:
#newdf4 = pd.DataFrame({r.name:r.df()[r.name] for r in response})
newdf2 = pd.concat([x.df() for x in response], axis=1)
#newdf2.columns
temp = newdf2

In [50]:
subset = ['#pmi', '#ConConf', '#BusConf','#CPI', '#PPI','#RetailSales']
#newdf2_ = temp[['#pmi', '#ConConf', '#BusConf','#CPI', '#PPI','#RetailSales']]
newdf2_ = temp[subset]
newdf2_['PERIOD_REFERENCE_DATE']= temp.iloc[:,0]

newdf2_
#subset

C:\blp\BQuant\environments\bqnt-1.281.1\lib\site-packages\bqnb\kernelapp.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # provided solely under the terms of the BFLP license agreement.


,#pmi,#ConConf,#BusConf,#CPI,#PPI,#RetailSales,PERIOD_REFERENCE_DATE
ID,,,,,,,
GB Country,54.2,-10.0,3.9,2.9,3.56473,0.7,2017-06-30
GB Country,55.5,-12.0,-4.9,2.6,3.27409,2.9,2017-07-31
GB Country,57.0,-10.0,9.8,2.6,3.26797,1.0,2017-08-31
GB Country,55.5,-9.0,-4.2,2.9,3.44827,2.4,2017-09-30
GB Country,56.6,-10.0,4.0,3.0,3.34572,1.6,2017-10-31
GB Country,58.2,-12.0,-5.0,3.0,2.86242,0.1,2017-11-30
GB Country,55.7,-13.0,9.8,3.1,3.13653,1.1,2017-12-31
GB Country,55.2,-9.0,0.6,3.0,3.12788,1.8,2018-01-31
GB Country,55.3,-10.0,-7.5,3.0,2.83623,2.0,2018-02-28


In [ ]:
newdf = df.reset_index()
newdf.head()

In [ ]:
newdf.dtypes

# Eco Example #3

In [51]:
letstring = """
let(
#CPILVL = znav(if(CPI <=3.5 AND CPI >=1.5,1,0).value);
#GDPZ= znav(if(last(zscore(GDP(PO=Range(-39q,0q))))>1,1,0).value);
#GDPLVL = znav(if(GDP >2.0,1,0).value);
#PMILVL = znav(if(PMI >50,1,-1).value);
#PMIZ= znav(if(last(zscore(PMI(PO=Range(-1m,0m))))>0,1,0).value);
#Reserves = znav(if(International_Reserves >50000000000,1,0).value);
#Score = (#CPILVL + #GDPLVL +#GDPZ+#PMILVL+#Reserves+#PMIZ);
)
"""

getstring = """
get(groupsort(#score),ID().country_name,#cpilvl,#gdplvl,#gdpz,#PMILVL,#Reserves,#PMIZ)
"""

country = "\'GB Country\'"


forstring = "for" + "(members(\'Asia\',type=\'Region\'))"

In [52]:
forstring

"for(members('Asia',type='Region'))"

In [53]:

request6 = letstring + getstring + forstring

In [54]:
response = bq.execute(request6)

df =bql.combined_df(response)    # newdf = response[0].df() if only one field
df.head()

,groupsort(#score),ID().country_name,#CPILVL,#GDPLVL,#GDPZ,#PMILVL,#Reserves,#PMIZ
ID,,,,,,,,
VN,3.0,Vietnam,1.0,1.0,1.0,-1.0,1.0,0.0
AZ,2.0,Azerbaijan,1.0,1.0,0.0,0.0,0.0,0.0
BD,2.0,Bangladesh,0.0,1.0,1.0,0.0,0.0,0.0
KH,2.0,Cambodia,1.0,1.0,0.0,0.0,0.0,0.0
HK,2.0,Hong Kong,1.0,0.0,0.0,0.0,1.0,0.0


# Technical Analysis Example #1

In [55]:
request7 = """
get(
RSI(PX_LAST(ca_adj=full),period=14)
)
for(members('SPX Index'))
"""


In [56]:
response = bq.execute(request7)

df =bql.combined_df(response)    # newdf = response[0].df() if only one field
df.head()

,DATE,"RSI(PX_LAST(ca_adj=full),period=14)"
ID,,
LYB UN Equity,2020-05-26,68.885562
AXP UN Equity,2020-05-26,57.967909
VZ UN Equity,2020-05-26,42.356438
AVGO UW Equity,2020-05-26,59.859745
BA UN Equity,2020-05-26,55.284106


<div style="text-align: right"><a href="#Sheet-Navigation">Return to Top ↑</a></div>

## BDH-Style Time Series Fetch using `bql.util` <a name="Z"></a>

In [1]:
import bqviz as bqv
import bql
bq = bql.Service()
from bql.util import get_time_series

In [8]:
help(get_time_series)

Help on function get_time_series in module bql.util.get_time_series:

get_time_series(tickers, fields, start, end=None, service=None)
    Return the time series of the fields for the specified tickers.
    
    Parameters
    ----------
    tickers: str, sequence
        A single ticker, or a sequence of tickers.
    fields: str, sequence
        A single field name, or a list of field names.
    start: datetime.date, str
        start date as :class:`datetime.date` or as a string in
        YYYY-MM-DD format.
    end: datetime.date, str (default: ``None``)
        end date as :class:`datetime.date` or as a string in
        YYYY-MM-DD format.
    service: :class:`bql.Service` (default: ``None``)
        the service to use to make the request, or ``None`` to use a default
        service.
    
    Returns
    -------
    :class:`pandas.DataFrame`
        A dataframe with a multi level index of ticker and date and
        field names as columns.



In [13]:
tickers = ['PTON US Equity', 'IBM US Equity']
start =  '2019-01-02'
end = '2020-02-28'
fields = ['PX_LAST', 'VWAP']

In [14]:
#data = get_time_series(tickers, fields, start, end)
data = get_time_series(tickers, fields, start, end)
#?get_time_series

In [15]:
#data.shape

In [16]:
data.sample(10)

CURRENCY  PX_LAST        VWAP
ID             DATE                                    
IBM US Equity  2019-11-13      USD   134.48  134.433735
PTON US Equity 2019-10-22      USD    21.16   21.387174
IBM US Equity  2020-01-22      USD   143.89  144.082495
               2020-01-02      USD   135.42  135.262049
               2020-01-27      USD   138.62  138.907700
PTON US Equity 2020-01-28      USD    31.38   31.102489
IBM US Equity  2019-11-19      USD   134.52  134.767307
PTON US Equity 2020-01-30      USD    32.26   31.957996
IBM US Equity  2019-12-04      USD   131.99  132.415554
               2019-05-01      USD   140.56  140.978460

In [6]:
pxlast = data.reset_index().pivot(index = 'DATE', columns ='ID',values ='PX_LAST')
pxlast

ID,SPTSX60 Index,SPX Index,SX5E Index
DATE,,,
2019-01-02,859.801,2510.03,2993.18
2019-01-03,851.451,2447.89,2954.66
2019-01-04,864.443,2531.94,3041.85
2019-01-07,868.547,2549.69,3033.64
2019-01-08,874.424,2574.41,3054.94
2019-01-09,885.913,2584.96,3070.24
2019-01-10,891.718,2596.64,3075.73
2019-01-11,893.385,2596.26,3070.04
2019-01-14,895.136,2582.61,3055.18


In [63]:
bqv.InteractiveLinePlot(pxlast).show()

In [64]:
bqv.InteractiveLinePlot?

Init signature: bqv.InteractiveLinePlot(df, fig_style='general', title='', colors=['#FFFFFF', '#1B84ED', '#CF7DFF', '#FF5A00', '#00D3D6', '#008616', '#B31D83', '#FF1E3E', '#FF9E24', '#63B2FF', '#30C030', '#9B016A', '#FD5D5D', '#AAAAAA', '#FBE360', '#00A5E3', '#35B56C', '#AC1BE8', '#007575', '#D20000', '#1E6AB4', '#89A4C5', '#626262'], hide_controls=False, tick_format=None, legend='default', y_min=None, y_max=None, num_ticks=None, y_label=None, x_label=None, is_subplot=False)
Docstring:     
Creates an interactive line plot for the provided dataframe.

Parameters
----------
df: Pandas dataframe
    The dataframe to be plotted.
title: string (default='')
    Optional title to be displayed on the figure.
colors: list of strings (default=BB_CLRS)
    Colors for the lines on the plot. Colors must be in hex format.
hide_controls: Boolean (default=False)
    Hides the defualt control widgets
legend: string (values: 'default', 'outside') or None
    Turns off the legend, or draws it outside of

## Options Example

In [65]:
request8 = """
get(
     OPEN_INT
)
for(
     filter(
            Options(['AAPL US EQUITY']), 
            PUT_CALL==CALL AND 
            EXPIRE_DT==2020-04-17 AND 
            OPEN_INT >0
))														
"""

In [66]:
response = bq.execute(request8)

df =bql.combined_df(response)    # newdf = response[0].df() if only one field
df.head()

BQL ERROR: No symbols met filter criteria in FILTER(Options(['AAPL US EQUITY']),PUT_CALL==CALL AND EXPIRE_DT==2020-04-17)., request_id=9536f03f07af443bb9b5ad2a09296d67 payload_id=None


ResponseError: No symbols met filter criteria in FILTER(Options(['AAPL US EQUITY']),PUT_CALL==CALL AND EXPIRE_DT==2020-04-17).

<div style="text-align: right"><a href="#Sheet-Navigation">Return to Top ↑</a></div>

## Object Model

In [ ]:
#import bql
#bq = bql.Service()

#fields
bql_item = bq.data.px_last()

#securities
bql_universe = 'UBER US Equity'

#build & execute request
bql_request = bql.Request(bql_universe, bql_item)
bql_response = bq.execute(bql_request)


In [ ]:
#display result
df = bql.combined_df(bql_response)
df